In [ ]:
import os
import torch
import numpy as np
import mediapipe as mp

# Directories
processed_folder = "/home/haggenmueller/asl_detection/machine_learning/datasets/own_dataset/processed_npy"
keypoints_folder = "/home/haggenmueller/asl_detection/machine_learning/datasets/own_dataset/keypoints_gpu"
os.makedirs(keypoints_folder, exist_ok=True)

# Initialize CUDA device (for tensor storage only, MediaPipe remains on CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🔹 Using device for tensor storage: {device}")

# Initialize MediaPipe Holistic (runs ONLY on CPU!)
mp_holistic = mp.solutions.holistic.Holistic(
    static_image_mode=False, model_complexity=1, smooth_landmarks=True, enable_segmentation=False
)

# Function to extract keypoints (Pose + Hands)
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33 * 3)
    left_hand = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21 * 3)
    right_hand = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21 * 3)
    
    return np.concatenate([pose, left_hand, right_hand])  # Total: 99 keypoints per frame

# Function to process a video (GPU for storage, CPU for MediaPipe)
def process_video(video_path, save_path):
    try:
        frames = np.load(video_path).astype(np.uint8)  # Convert to uint8

        keypoints_seq = []
        for frame in frames:
            frame_np = frame if isinstance(frame, np.ndarray) else frame.cpu().numpy()
            if frame_np.shape[0] == 3 and frame_np.shape[-1] != 3:
                frame_np = frame_np.transpose(1, 2, 0)  # [3, 224, 224] → [224, 224, 3]

            results = mp_holistic.process(frame_np)
            keypoints = extract_keypoints(results)
            keypoints_seq.append(keypoints)

        np.save(save_path, np.array(keypoints_seq))
        return True

    except Exception as e:
        print(f"❌ Error processing {video_path}: {e}")
        return False

# Process all videos
for video_file in os.listdir(processed_folder):
    if not video_file.endswith(".npy"):
        continue

    video_path = os.path.join(processed_folder, video_file)
    keypoint_path = os.path.join(keypoints_folder, video_file)

    if os.path.exists(keypoint_path):
        print(f"⏭️  Skipping {video_file} (already processed)")
        continue

    success = process_video(video_path, keypoint_path)

    if success:
        print(f"✅ Keypoints saved for {video_file}")

print("🚀 Keypoint extraction completed! (Processed only missing files)")